In [1]:
import mysql.connector
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker('es_ES')

In [3]:
import pyodbc

# Conexión a SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost\SQLEXPRESS;'
    'DATABASE=fashion_retail;'
    'UID=test;'
    'PWD=pepe3915-'
)
cursor = conn.cursor()


In [5]:

# ---------- CONFIGURACIÓN ----------
NUM_CLIENTES = 10
NUM_PROVEEDORES = 5
NUM_CATEGORIAS = 3
NUM_MARCAS = 5
NUM_PRODUCTOS = 20
NUM_VENTAS = 1000
NUM_COMPRAS = 590
# -----------------------------------

In [8]:
def insert_categorias():
    categorias = ['Ropa', 'Calzado', 'Accesorios']
    for i, nombre in enumerate(categorias, start=1):
        cursor.execute(
            "INSERT INTO Categorias (ID_Categoria, Nombre_Categoria, Descripcion) VALUES (?, ?, ?)",
            (i, nombre, f"{nombre} de moda para todas las edades")
        )

def insert_marcas():
    for i in range(1, NUM_MARCAS + 1):
        cursor.execute(
            "INSERT INTO Marcas (ID_Marca, Nombre_Marca, Pais_Origen) VALUES (?, ?, ?)",
            (i, fake.company(), fake.country())
        )

def insert_productos():
    for i in range(1, NUM_PRODUCTOS + 1):
        categoria_id = random.randint(1, NUM_CATEGORIAS)
        marca_id = random.randint(1, NUM_MARCAS)
        precio = round(random.uniform(10, 100), 2)
        costo = round(precio * 0.6, 2)
        stock = random.randint(0, 100)
        talla = random.choice(['S', 'M', 'L', 'XL', 'N/A'])
        color = random.choice(['Negro', 'Blanco', 'Rojo', 'Azul', 'Verde'])
        genero = random.choice(['Hombre', 'Mujer', 'Unisex'])
        fecha_ingreso = fake.date_between(start_date='-90d', end_date='today')
        cursor.execute(
            "INSERT INTO Productos (ID_Producto, Nombre, Descripcion, ID_Categoria, ID_Marca, Precio, Costo, Stock, Talla, Color, Genero, Fecha_Ingreso) "
            "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (i, fake.word().capitalize(), fake.sentence(), categoria_id, marca_id, precio, costo, stock, talla, color, genero, fecha_ingreso)
        )

def insert_clientes():
    for i in range(1, NUM_CLIENTES + 1):
        cursor.execute(
            "INSERT INTO Clientes (ID_Cliente, Nombre, Apellido, Email, Telefono, Direccion, Fecha_Registro) VALUES (?, ?, ?, ?, ?, ?, ?)",
            (i, fake.first_name(), fake.last_name(), fake.email(), fake.phone_number(), fake.address(), fake.date_this_year())
        )

def insert_proveedores():
    for i in range(1, NUM_PROVEEDORES + 1):
        cursor.execute(
            "INSERT INTO Proveedores (ID_Proveedor, Nombre_Empresa, Contacto, Email, Telefono, Direccion) VALUES (?, ?, ?, ?, ?, ?)",
            (i, fake.company(), fake.name(), fake.company_email(), fake.phone_number(), fake.address())
        )

def insert_compras():
    for i in range(1, NUM_COMPRAS + 1):
        proveedor_id = random.randint(1, NUM_PROVEEDORES)
        fecha = fake.date_between(start_date='-60d', end_date='today')
        total = 0
        cursor.execute(
            "INSERT INTO Compras (ID_Compra, ID_Proveedor, Fecha_Compra, Total_Compra) VALUES (?, ?, ?, ?)",
            (i, proveedor_id, fecha, total)
        )
        
        detalles_count = random.randint(1, 3)
        for d in range(1, detalles_count + 1):
            producto_id = random.randint(1, NUM_PRODUCTOS)
            cantidad = random.randint(1, 10)
            precio_unit = round(random.uniform(5, 50), 2)
            subtotal = round(cantidad * precio_unit, 2)
            total += subtotal
            cursor.execute(
                "INSERT INTO Detalle_Compras (ID_Detalle_Compra, ID_Compra, ID_Producto, Cantidad, Precio_Unitario, Subtotal) VALUES (?, ?, ?, ?, ?, ?)",
                (i*10 + d, i, producto_id, cantidad, precio_unit, subtotal)
            )
        cursor.execute(
            "UPDATE Compras SET Total_Compra = ? WHERE ID_Compra = ?",
            (total, i)
        )

def insert_ventas():
    for i in range(1, NUM_VENTAS + 1):
        cliente_id = random.randint(1, NUM_CLIENTES)
        fecha = fake.date_between(start_date='-30d', end_date='today')
        total = 0
        metodo_pago = random.choice(['Efectivo', 'Tarjeta', 'PayPal'])
        cursor.execute(
            "INSERT INTO Ventas (ID_Venta, ID_Cliente, Fecha_Venta, Total_Venta, Metodo_Pago) VALUES (?, ?, ?, ?, ?)",
            (i, cliente_id, fecha, total, metodo_pago)
        )
        
        detalles_count = random.randint(1, 3)
        for d in range(1, detalles_count + 1):
            producto_id = random.randint(1, NUM_PRODUCTOS)
            cantidad = random.randint(1, 2)
            cursor.execute("SELECT Precio FROM Productos WHERE ID_Producto = ?", (producto_id,))
            precio_unit = cursor.fetchone()[0]
            subtotal = round(precio_unit * cantidad, 2)
            total += subtotal
            cursor.execute(
                "INSERT INTO Detalle_Ventas (ID_Detalle, ID_Venta, ID_Producto, Cantidad, Precio_Unitario, Subtotal) VALUES (?, ?, ?, ?, ?, ?)",
                (i*10 + d, i, producto_id, cantidad, precio_unit, subtotal)
            )
        cursor.execute(
            "UPDATE Ventas SET Total_Venta = ? WHERE ID_Venta = ?",
            (total, i)
        )

In [9]:
# Ejecutar todo
insert_categorias()
insert_marcas()
insert_productos()
insert_clientes()
insert_proveedores()
insert_compras()
insert_ventas()

# Confirmar cambios
conn.commit()
cursor.close()
conn.close()
print("Datos insertados correctamente.")

Datos insertados correctamente.
